## Task 1 Number of alignments

Чтобы посчитать общее количество выравниваний двух последовательностей длины n и m (для удобства предположим, что n < m), посчитаем сколько возможно выравниваний при которых будут совпадения/несовпадения k символов. Для этого нужно выбрать k позиций последовательности длины m, на которых произойдут эти совпадения. Это можно сделать $$\binom{m}{k}$$ способами (это сочетания и не перестановки, так как порядок строго задан исходной последовательностью). Теперь, когда мы определили k позиций, в которых не будет гэпа, надо определить, какие элементы другой последовательности будут им соответствовать. Так как порядок опять фиксирован, надо выбрать какие k символов встанут в эти позиции, ясно что это количество сочетаний - $$\binom{n}{k}$$
То есть количество выравниваний с m - k гэпами будет равно $$\binom{n}{k}\binom{m}{k}$$
Так как k может быть любым, просуммируем по k $$0 <= k <= min(n, m)$$

В итоге получаем количество выравниваний:$$N = \sum_{k=0}^{n}\binom{n}{k}\binom{m}{k}$$
Заметим, что выражение под суммой симметрично относительно m и n, а $$\binom{n}{k} = 0, k > n$$, это значит, что условие n < m больше не необходимо.

## Task 2 Global alignment

In [1]:
import numpy as np
from collections import namedtuple

In [2]:
Score = namedtuple('Score', ('match', 'mismatch', 'gap'))

In [3]:
def solve(N, M, s):
    n = len(N)
    m = len(M)
    d = np.zeros((m + 1, n + 1))
    d[0] = np.arange(n + 1) * s.gap
    for i in range(m + 1):
        d[i][0] = i * s.gap

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[i, j] = max(
                d[i - 1, j] + s.gap, 
                d[i, j - 1] + s.gap, 
                d[i - 1, j - 1] + (s.match if N[j - 1] == M[i - 1] else s.mismatch)
            )
    return d

In [4]:
def construct_alignment(N, M, d, s):
    a = ''
    b = ''
    i, j = len(M), len(N)
    while i > 0 and j > 0:
        if d[i][j] == d[i - 1][j - 1] + (s.match if N[j - 1] == M[i - 1] else s.mismatch):
            a = M[i - 1] + a
            b = N[j - 1] + b
            i -= 1
            j -= 1
        elif d[i][j] == d[i][j - 1] + s.gap:
            a = '-' + a
            b = N[j - 1] + b
            j -= 1
        else:
            a = M[i - 1] + a
            b = '-' + b
            i -= 1

    if j == 0:
        a = M[:i] + a
        b = '-' * i + b

    else:
        a = '-' * j + a
        b = N[:j] + b
    return a, b

#### Test 1

In [5]:
N = 'TTAGGATCAAT'
M = 'AATAGGGCAATT'

# Match score
match = 1
# Gap penalty
gap = -1
# Mismatch penalty
mismatch = -1

scores = Score(match, mismatch, gap)

In [6]:
d = solve(N, M, scores)
print(d)

[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.]
 [ -1.  -1.  -2.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.]
 [ -2.  -2.  -2.  -1.  -2.  -3.  -2.  -3.  -4.  -5.  -6.  -7.]
 [ -3.  -1.  -1.  -2.  -2.  -3.  -3.  -1.  -2.  -3.  -4.  -5.]
 [ -4.  -2.  -2.   0.  -1.  -2.  -2.  -2.  -2.  -1.  -2.  -3.]
 [ -5.  -3.  -3.  -1.   1.   0.  -1.  -2.  -3.  -2.  -2.  -3.]
 [ -6.  -4.  -4.  -2.   0.   2.   1.   0.  -1.  -2.  -3.  -3.]
 [ -7.  -5.  -5.  -3.  -1.   1.   1.   0.  -1.  -2.  -3.  -4.]
 [ -8.  -6.  -6.  -4.  -2.   0.   0.   0.   1.   0.  -1.  -2.]
 [ -9.  -7.  -7.  -5.  -3.  -1.   1.   0.   0.   2.   1.   0.]
 [-10.  -8.  -8.  -6.  -4.  -2.   0.   0.  -1.   1.   3.   2.]
 [-11.  -9.  -7.  -7.  -5.  -3.  -1.   1.   0.   0.   2.   4.]
 [-12. -10.  -8.  -8.  -6.  -4.  -2.   0.   0.  -1.   1.   3.]]


In [7]:
a, b = construct_alignment(N, M, d, scores)
print('Выравнивание:')
print(a)
print(b)

print('Вес выравнивания:', d[-1, -1])

Выравнивание:
AATAGG-GCAATT
-TTAGGATCAA-T
Вес выравнивания: 3.0


#### Test 2

In [8]:
N = 'TTAGGCAT'
M = 'AATGGCAA'

# Match score
match = 1
# Gap penalty
gap = -0.499
# Mismatch penalty
mismatch = -1

scores = Score(match, mismatch, gap)

In [9]:
d = solve(N, M, scores)
print(d)

[[-0.000e+00 -4.990e-01 -9.980e-01 -1.497e+00 -1.996e+00 -2.495e+00
  -2.994e+00 -3.493e+00 -3.992e+00]
 [-4.990e-01 -9.980e-01 -1.497e+00  2.000e-03 -4.970e-01 -9.960e-01
  -1.495e+00 -1.994e+00 -2.493e+00]
 [-9.980e-01 -1.497e+00 -1.996e+00 -4.970e-01 -9.960e-01 -1.495e+00
  -1.994e+00 -4.950e-01 -9.940e-01]
 [-1.497e+00  2.000e-03 -4.970e-01 -9.960e-01 -1.495e+00 -1.994e+00
  -2.493e+00 -9.940e-01  5.050e-01]
 [-1.996e+00 -4.970e-01 -9.960e-01 -1.495e+00  4.000e-03 -4.950e-01
  -9.940e-01 -1.493e+00  6.000e-03]
 [-2.495e+00 -9.960e-01 -1.495e+00 -1.994e+00 -4.950e-01  1.004e+00
   5.050e-01  6.000e-03 -4.930e-01]
 [-2.994e+00 -1.495e+00 -1.994e+00 -2.493e+00 -9.940e-01  5.050e-01
   2.004e+00  1.505e+00  1.006e+00]
 [-3.493e+00 -1.994e+00 -2.493e+00 -9.940e-01 -1.493e+00  6.000e-03
   1.505e+00  3.004e+00  2.505e+00]
 [-3.992e+00 -2.493e+00 -2.992e+00 -1.493e+00 -1.992e+00 -4.930e-01
   1.006e+00  2.505e+00  2.006e+00]]


In [10]:
a, b = construct_alignment(N, M, d, scores)
print('Выравнивание:')
print(a)
print(b)

print('Вес выравнивания:', d[-1, -1])

Выравнивание:
AA-T-GGCAA-
--TTAGGC-AT
Вес выравнивания: 2.006


## Task 3 Alignment with weight matrix

In [11]:
import pandas as pd

In [12]:
def weight_func(wm):
    # wm - weight matrix
    def weight(v, w):
        if v == '-' or w == '-':
            return g
        else:
            return wm[v][w]
    
    return weight

In [13]:
def solve_wm(N, M, f):
    # f - function that returns weights
    n = len(N)
    m = len(M)
    d = np.zeros((m + 1, n + 1))
    d[0] = np.arange(n + 1) * f('-', '-')
    for i in range(m + 1):
        d[i][0] = i * f('-', '-')

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[i, j] = max(
                d[i - 1, j] + f('-', M[i - 1]), 
                d[i, j - 1] + f(N[j - 1], '-'), 
                d[i - 1, j - 1] + f(N[j - 1], M[i - 1])
            )
    return d

In [14]:
def construct_alignment_wm(N, M, d, f):
    a = ''
    b = ''
    i, j = len(M), len(N)
    while i > 0 and j > 0:
        if d[i][j] == d[i - 1][j - 1] + f(N[j - 1], M[i - 1]):
            a = M[i - 1] + a
            b = N[j - 1] + b
            i -= 1
            j -= 1
        elif d[i][j] == d[i][j - 1] + f(N[j - 1], '-'):
            a = '-' + a
            b = N[j - 1] + b
            j -= 1
        else:
            a = M[i - 1] + a
            b = '-' + b
            i -= 1

    if j == 0:
        a = M[:i] + a
        b = '-' * i + b

    else:
        a = '-' * j + a
        b = N[:j] + b
    return a, b

In [15]:
N = 'TTAGGATCAAT'
M = 'AATAGGGCAATT'

# Gap penalty
g = -1.
# weight matrix
wm1 = pd.DataFrame(
    {
        'A':   [1., -1., -0.5, -1., g], 
        'T':   [-1., 1., -1., -0.5, g], 
        'G':   [-0.5, -1., 1., -1., g], 
        'C':   [-1., -0.5, -1., 1., g],
        'gap': [g, g, g, g, g],
    }, 
    index=['A', 'T', 'G', 'C', 'gap']
)

In [16]:
weights1 = weight_func(wm1)
d = solve_wm(N, M, weights1)
print(d)
a, b = construct_alignment_wm(N, M, d, weights1)
print('Выравнивание:')
print(a)
print(b)

print('Вес выравнивания:', d[-1, -1])

[[ -0.   -1.   -2.   -3.   -4.   -5.   -6.   -7.   -8.   -9.  -10.  -11. ]
 [ -1.   -1.   -2.   -1.   -2.   -3.   -4.   -5.   -6.   -7.   -8.   -9. ]
 [ -2.   -2.   -2.   -1.   -1.5  -2.5  -2.   -3.   -4.   -5.   -6.   -7. ]
 [ -3.   -1.   -1.   -2.   -2.   -2.5  -3.   -1.   -2.   -3.   -4.   -5. ]
 [ -4.   -2.   -2.    0.   -1.   -2.   -1.5  -2.   -2.   -1.   -2.   -3. ]
 [ -5.   -3.   -3.   -1.    1.    0.   -1.   -2.   -3.   -2.   -1.5  -2.5]
 [ -6.   -4.   -4.   -2.    0.    2.    1.    0.   -1.   -2.   -2.5  -2.5]
 [ -7.   -5.   -5.   -3.   -1.    1.    1.5   0.5  -0.5  -1.5  -2.5  -3.5]
 [ -8.   -6.   -5.5  -4.   -2.    0.    0.5   1.    1.5   0.5  -0.5  -1.5]
 [ -9.   -7.   -6.5  -4.5  -3.   -1.    1.    0.    0.5   2.5   1.5   0.5]
 [-10.   -8.   -7.5  -5.5  -4.   -2.    0.    0.   -0.5   1.5   3.5   2.5]
 [-11.   -9.   -7.   -6.5  -5.   -3.   -1.    1.    0.    0.5   2.5   4.5]
 [-12.  -10.   -8.   -7.5  -6.   -4.   -2.    0.    0.5  -0.5   1.5   3.5]]
Выравнивание:
AATAGGG-CA

In [17]:
# Gap penalty
g = -1.
# weight matrix
wm2 = pd.DataFrame(
    {
        'A':   [1., -3., -0.5, -1., g], 
        'T':   [-3., 1., -1., -0.5, g], 
        'G':   [-0.5, -1., 1., -1., g], 
        'C':   [-1., -0.5, -1., 1., g],
        'gap': [g, g, g, g, g],
    }, 
    index=['A', 'T', 'G', 'C', 'gap']
)

In [18]:
weights2 = weight_func(wm2)
d = solve_wm(N, M, weights2)
print(d)
a, b = construct_alignment_wm(N, M, d, weights2)
print('Выравнивание:')
print(a)
print(b)

print('Вес выравнивания:', d[-1, -1])

[[ -0.   -1.   -2.   -3.   -4.   -5.   -6.   -7.   -8.   -9.  -10.  -11. ]
 [ -1.   -2.   -3.   -1.   -2.   -3.   -4.   -5.   -6.   -7.   -8.   -9. ]
 [ -2.   -3.   -4.   -2.   -1.5  -2.5  -2.   -3.   -4.   -5.   -6.   -7. ]
 [ -3.   -1.   -2.   -3.   -2.5  -2.5  -3.   -1.   -2.   -3.   -4.   -5. ]
 [ -4.   -2.   -3.   -1.   -2.   -3.   -1.5  -2.   -2.   -1.   -2.   -3. ]
 [ -5.   -3.   -3.   -2.    0.   -1.   -2.   -2.5  -3.   -2.   -1.5  -2.5]
 [ -6.   -4.   -4.   -3.   -1.    1.    0.   -1.   -2.   -3.   -2.5  -2.5]
 [ -7.   -5.   -5.   -4.   -2.    0.    0.5  -0.5  -1.5  -2.5  -3.5  -3.5]
 [ -8.   -6.   -5.5  -5.   -3.   -1.   -0.5   0.    0.5  -0.5  -1.5  -2.5]
 [ -9.   -7.   -6.5  -4.5  -4.   -2.    0.   -1.   -0.5   1.5   0.5  -0.5]
 [-10.   -8.   -7.5  -5.5  -5.   -3.   -1.   -2.   -1.5   0.5   2.5   1.5]
 [-11.   -9.   -7.   -6.5  -6.   -4.   -2.    0.   -1.   -0.5   1.5   3.5]
 [-12.  -10.   -8.   -7.5  -7.   -5.   -3.   -1.   -0.5  -1.5   0.5   2.5]]
Выравнивание:
AA-TAGGG-C

## Task 4 Local alignment

In [19]:
def solve_local(N, M, s):
    n = len(N)
    m = len(M)
    
    d = np.zeros((m + 1, n + 1))
    d[0] = np.arange(n + 1) * s.gap
    for i in range(m + 1):
        d[i][0] = i * s.gap

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[i, j] = max(
                0,
                d[i - 1, j] + s.gap, 
                d[i, j - 1] + s.gap, 
                d[i - 1, j - 1] + (s.match if N[j - 1] == M[i - 1] else s.mismatch)
            )
    return d

In [20]:
def construct_local_alignment(N, M, d, s):
    a = ''
    b = ''
    i, j = np.unravel_index(np.argmax(d, axis=None), d.shape)
    while i > 0 and j > 0:
        if d[i][j] == 0:
            return a, b
        if d[i][j] == d[i - 1][j - 1] + (s.match if N[j - 1] == M[i - 1] else s.mismatch):
            a = M[i - 1] + a
            b = N[j - 1] + b
            i -= 1
            j -= 1
        elif d[i][j] == d[i][j - 1] + s.gap:
            a = '-' + a
            b = N[j - 1] + b
            j -= 1
        else:
            a = M[i - 1] + a
            b = '-' + b
            i -= 1

    if j == 0:
        a = M[:i] + a
        b = '-' * i + b

    else:
        a = '-' * j + a
        b = N[:j] + b
    return a, b

In [21]:
N = 'TTAGGATCAAT'
M = 'AATAGGGCAATT'

# Match score
match = 1
# Gap penalty
gap = -1
# Mismatch penalty
mismatch = -1

scores = Score(match, mismatch, gap)

In [22]:
d = solve_local(N, M, scores)
print(d)
a, b = construct_local_alignment(N, M, d, scores)
print('Выравнивание:')
print(a)
print(b)

print('Вес выравнивания:', np.max(d, axis=None))

[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.   0.   0.   1.   0.   0.   1.   0.   0.   1.   1.   0.]
 [ -3.   0.   1.   0.   0.   0.   0.   2.   1.   0.   0.   2.]
 [ -4.   0.   0.   2.   1.   0.   1.   1.   1.   2.   1.   1.]
 [ -5.   0.   0.   1.   3.   2.   1.   0.   0.   1.   1.   0.]
 [ -6.   0.   0.   0.   2.   4.   3.   2.   1.   0.   0.   0.]
 [ -7.   0.   0.   0.   1.   3.   3.   2.   1.   0.   0.   0.]
 [ -8.   0.   0.   0.   0.   2.   2.   2.   3.   2.   1.   0.]
 [ -9.   0.   0.   1.   0.   1.   3.   2.   2.   4.   3.   2.]
 [-10.   0.   0.   1.   0.   0.   2.   2.   1.   3.   5.   4.]
 [-11.   0.   1.   0.   0.   0.   1.   3.   2.   2.   4.   6.]
 [-12.   0.   1.   0.   0.   0.   0.   2.   2.   1.   3.   5.]]
Выравнивание:
TAGG-GCAAT
TAGGATCAAT
Вес выравнивания: 6.0


#### Test 1

In [23]:
M = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
N = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'


match = 1
gap = -1
mismatch = -1
scores = Score(match, mismatch, gap)

d_local = solve_local(N, M, scores)
a_local, b_local = construct_local_alignment(N, M, d_local, scores)
print('Локальное выравнивание:')
print(a_local)
print(b_local)
d = solve(N, M, scores)
a, b = construct_alignment(N, M, d, scores)
print('Глобальное выравнивание:')
print(a)
print(b)
print('Выравнивания различаются на локальном участке')

Локальное выравнивание:
CAGTTATGTCAG
CAGTTATGTCAG
Глобальное выравнивание:
---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC
Выравнивания различаются на локальном участке


## Task 5 Affine gap alignment

In [24]:
def solve_ag(N, M, s):
    n = len(N)
    m = len(M)
    
    # deletion (upper) layer (vertical movement) 
    v = np.zeros((m + 1, n + 1))
    v[0] = np.ones(n + 1) * (manhattan_scores.gap + manhattan_scores.open_gap) * 100
    for i in range(m + 1):
        v[i][0] = i * manhattan_scores.gap + manhattan_scores.open_gap
        
    # substitution middle layer (dioganal movement) 
    d = np.zeros((m + 1, n + 1))
    d[0] = np.arange(n + 1) * manhattan_scores.gap + manhattan_scores.open_gap
    for i in range(m + 1):
        d[i][0] = i * manhattan_scores.gap + manhattan_scores.open_gap
        
    # insertion (lower) layer (horizontal movement)
    h = np.zeros((m + 1, n + 1))
    h[0] = np.arange(n + 1) * manhattan_scores.gap + manhattan_scores.open_gap
    for i in range(m + 1):
        h[i][0] = (manhattan_scores.gap + manhattan_scores.open_gap) * 100
        
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            v[i, j] = max(
                v[i - 1, j] + s.gap,
                d[i - 1, j] + s.open_gap + s.gap
            )
            h[i, j] = max(
                h[i, j - 1] + s.gap,
                d[i, j - 1] + s.open_gap + s.gap
            )
            d[i, j] = max(
                v[i, j], 
                h[i, j], 
                d[i - 1, j - 1] + (s.match if N[j - 1] == M[i - 1] else s.mismatch)
            )
    return d, v, h

In [25]:
def construct_alignment_ag(N, M, tables, s):
    a = ''
    b = ''
    i, j = len(M), len(N)
    d, v, h = tables
    while i > 0 and j > 0:
        if d[i][j] == d[i - 1][j - 1] + (s.match if N[j - 1] == M[i - 1] else s.mismatch):
            a = M[i - 1] + a
            b = N[j - 1] + b
            i -= 1
            j -= 1
        elif d[i][j] == h[i][j]:
            a = '-' + a
            b = N[j - 1] + b
            j -= 1
        else:
            a = M[i - 1] + a
            b = '-' + b
            i -= 1

    if j == 0:
        a = M[:i] + a
        b = '-' * i + b

    else:
        a = '-' * j + a
        b = N[:j] + b
    return a, b

In [26]:
Mscores = namedtuple('Mscores', ('match', 'mismatch', 'open_gap', 'gap'))
manhattan_scores = Mscores(1, -1, 0, -1)

N = 'TTAGGATCAAT'
M = 'AATAGGGCAATT'

d, v, h = solve_ag(N, M, manhattan_scores)
print(d)

[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.]
 [ -1.  -1.  -2.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.]
 [ -2.  -2.  -2.  -1.  -2.  -3.  -2.  -3.  -4.  -5.  -6.  -7.]
 [ -3.  -1.  -1.  -2.  -2.  -3.  -3.  -1.  -2.  -3.  -4.  -5.]
 [ -4.  -2.  -2.   0.  -1.  -2.  -2.  -2.  -2.  -1.  -2.  -3.]
 [ -5.  -3.  -3.  -1.   1.   0.  -1.  -2.  -3.  -2.  -2.  -3.]
 [ -6.  -4.  -4.  -2.   0.   2.   1.   0.  -1.  -2.  -3.  -3.]
 [ -7.  -5.  -5.  -3.  -1.   1.   1.   0.  -1.  -2.  -3.  -4.]
 [ -8.  -6.  -6.  -4.  -2.   0.   0.   0.   1.   0.  -1.  -2.]
 [ -9.  -7.  -7.  -5.  -3.  -1.   1.   0.   0.   2.   1.   0.]
 [-10.  -8.  -8.  -6.  -4.  -2.   0.   0.  -1.   1.   3.   2.]
 [-11.  -9.  -7.  -7.  -5.  -3.  -1.   1.   0.   0.   2.   4.]
 [-12. -10.  -8.  -8.  -6.  -4.  -2.   0.   0.  -1.   1.   3.]]


In [27]:
tables = d, v, h
a, b = construct_alignment_ag(N, M, tables, manhattan_scores)
print(a)
print(b)

AATAGG-GCAATT
-TTAGGATCAA-T


#### Test 1

In [28]:
M = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
N = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

# match, mismatch, open gap, continue gap
manhattan_scores = Mscores(1, -1, 0, -1)
d, v, h = solve_ag(N, M, manhattan_scores)
print(d)
tables = d, v, h
a, b = construct_alignment_ag(N, M, tables, manhattan_scores)
print(a)
print(b)

[[  0.  -1.  -2. ... -34. -35. -36.]
 [ -1.  -1.  -2. ... -32. -33. -34.]
 [ -2.  -2.  -2. ... -30. -31. -32.]
 ...
 [-34. -32. -30. ...   2.   2.   1.]
 [-35. -33. -31. ...   1.   1.   1.]
 [-36. -34. -32. ...   0.   0.   2.]]
---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC


#### Test 2

In [29]:
M = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
N = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

# match, mismatch, open gap, continue gap
manhattan_scores = Mscores(1, -1, -100, -0.01)
d, v, h = solve_ag(N, M, manhattan_scores)
print(d)
tables = d, v, h
a, b = construct_alignment_ag(N, M, tables, manhattan_scores)
print(a)
print(b)

[[-100.   -100.01 -100.02 ... -100.34 -100.35 -100.36]
 [-100.01 -101.   -101.01 ... -101.33  -99.34 -101.35]
 [-100.02 -101.01 -102.   ... -102.32 -102.33  -98.34]
 ...
 [-100.34 -101.33 -100.32 ... -120.   -122.01 -120.02]
 [-100.35  -99.34 -100.33 ... -116.01 -121.   -123.01]
 [-100.36 -101.35 -100.34 ... -128.02 -117.01 -120.  ]]
TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC
AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC


#### Test 3

In [30]:
M = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
N = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

# match, mismatch, open gap, continue gap
manhattan_scores = Mscores(1, -1, 0.5, -0.3)
d, v, h = solve_ag(N, M, manhattan_scores)
print(d)
tables = d, v, h
a, b = construct_alignment_ag(N, M, tables, manhattan_scores)
print(a)
print(b)

[[  0.5   0.2  -0.1 ...  -9.7 -10.  -10.3]
 [  0.2  19.7  19.9 ...  26.3  26.5  26.7]
 [ -0.1  19.9  20.1 ...  27.1  27.3  27.5]
 ...
 [ -9.7  26.3  27.1 ...  45.5  45.7  45.9]
 [-10.   26.5  27.3 ...  45.7  45.9  46.1]
 [-10.3  26.7  27.5 ...  45.9  46.1  46.9]]
T-----CC-CA-GT--TATG-TCAGGGGACACGAGC--ATG-CAGAGA-C
-AATTGCCGC-CGTCGT-T-TTCA---G---C-AG-TTATGTC--AGATC
